In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')

In [39]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 무

In [40]:
rad = pd.read_csv("C:/Users/keogu/Desktop/빅데이터 자료분석/new data/무.csv",parse_dates=['SALEDATE'])

In [41]:
rad

,PUM_NM,LV_NM,TOT_AMT,SAN_NM,SALEDATE,CMP_NM,DAN_NM,WHSAL_NM,SIZE_NM,COST,POJ_NM,TOT_QTY,QTY,KIND_NM,DANQ
0,무,.,1456000.0,제주도 서귀포시,2014-01-03,서울청과,kg,서울가락도매,.,5200.0,.,5040.0,280.0,기타무,18.0
1,무,특,1344000.0,제주도 서귀포시,2014-01-03,한국청과,kg,서울가락도매,.,4800.0,PE대,5040.0,280.0,기타무,18.0
2,무,특,1400000.0,제주도 제주시,2014-01-03,한국청과,kg,서울가락도매,.,5000.0,PE대,5040.0,280.0,기타무,18.0
3,무,특,1344000.0,제주도 제주시,2014-01-03,한국청과,kg,서울가락도매,.,4800.0,PE대,5040.0,280.0,기타무,18.0
4,무,특,1484000.0,제주도 서귀포시,2014-01-03,한국청과,kg,서울가락도매,.,5300.0,PE대,5040.0,280.0,기타무,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883767,무,특,64000.0,강원 횡성군,2021-10-30,대인농산,kg,인천구월도매,10개,8000.0,상자,160.0,8.0,가을무,20.0
1883768,무,특,336000.0,강원 횡성군,2021-10-30,대인농산,kg,인천구월도매,9개,7000.0,상자,960.0,48.0,가을무,20.0
1883769,무,특,30000.0,강원 횡성군,2021-10-30,대인농산,kg,인천구월도매,8개,7500.0,상자,80.0,4.0,가을무,20.0
1883770,무,특,93600.0,강원 횡성군,2021-10-30,대인농산,kg,인천구월도매,.,7200.0,상자,260.0,13.0,가을무,20.0


In [42]:
rad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883772 entries, 0 to 1883771
Data columns (total 15 columns):
 #   Column    Dtype         
---  ------    -----         
 0   PUM_NM    object        
 1   LV_NM     object        
 2   TOT_AMT   float64       
 3   SAN_NM    object        
 4   SALEDATE  datetime64[ns]
 5   CMP_NM    object        
 6   DAN_NM    object        
 7   WHSAL_NM  object        
 8   SIZE_NM   object        
 9   COST      float64       
 10  POJ_NM    object        
 11  TOT_QTY   float64       
 12  QTY       float64       
 13  KIND_NM   object        
 14  DANQ      float64       
dtypes: datetime64[ns](1), float64(5), object(9)
memory usage: 215.6+ MB


## 필요없는 컬럼 삭제  
**필요없는 칼럼**
- PUM_NM: 품목
- LV_NM: 등급  
- CMP_NM: 법인
- DAN_NM: 단위
- WHSAL_NM: 도매시장 
- SIZE_NM: 크기
- COST: 단가
- POJ_NM: 포장 
- QTY: 물량
- KIND_NM: 품종 
- DANQ: 단위물량

In [43]:
rad.drop(columns=['CMP_NM','WHSAL_NM','POJ_NM','KIND_NM','PUM_NM','LV_NM','SIZE_NM','QTY','DANQ','COST','DAN_NM'], axis=1, inplace=True)

In [44]:
rad

,TOT_AMT,SAN_NM,SALEDATE,TOT_QTY
0,1456000.0,제주도 서귀포시,2014-01-03,5040.0
1,1344000.0,제주도 서귀포시,2014-01-03,5040.0
2,1400000.0,제주도 제주시,2014-01-03,5040.0
3,1344000.0,제주도 제주시,2014-01-03,5040.0
4,1484000.0,제주도 서귀포시,2014-01-03,5040.0
...,...,...,...,...
1883767,64000.0,강원 횡성군,2021-10-30,160.0
1883768,336000.0,강원 횡성군,2021-10-30,960.0
1883769,30000.0,강원 횡성군,2021-10-30,80.0
1883770,93600.0,강원 횡성군,2021-10-30,260.0


**남은 컬럼**
- TOT_AMT: 총 금액
- SAN_NM: 산지
- SALEDATE: 경락 일자   
- TOT_QTY:총 물량

## 총 금액 / 총 물량 
- 음수거나 0인 값 제거(거래 취소)
- 이상치 제거(?)

In [45]:
rad[(rad['TOT_AMT'] <= 0)|(rad['TOT_QTY'] <= 0)]

,TOT_AMT,SAN_NM,SALEDATE,TOT_QTY
151,-868000.0,제주도 제주시,2014-01-03,-5040.0
342,-1624000.0,제주도 제주시,2014-01-06,-5040.0
344,-1808400.0,전라북도 부안군,2014-01-06,-9864.0
436,-1624000.0,제주도 제주시,2014-01-06,-5220.0
664,-1115200.0,제주도 제주시,2014-01-08,-4896.0
...,...,...,...,...
1883171,0.0,경기도 용인시,2021-10-30,40.0
1883292,0.0,강원도 홍천군,2021-10-30,20.0
1883294,0.0,강원도 홍천군,2021-10-30,20.0
1883403,0.0,강원도 춘천시,2021-10-30,20.0


In [46]:
drop_index = rad[(rad['TOT_AMT'] <= 0)|(rad['TOT_QTY'] <= 0)].index
rad.drop(index = drop_index, axis=0, inplace=True)

In [47]:
rad

,TOT_AMT,SAN_NM,SALEDATE,TOT_QTY
0,1456000.0,제주도 서귀포시,2014-01-03,5040.0
1,1344000.0,제주도 서귀포시,2014-01-03,5040.0
2,1400000.0,제주도 제주시,2014-01-03,5040.0
3,1344000.0,제주도 제주시,2014-01-03,5040.0
4,1484000.0,제주도 서귀포시,2014-01-03,5040.0
...,...,...,...,...
1883767,64000.0,강원 횡성군,2021-10-30,160.0
1883768,336000.0,강원 횡성군,2021-10-30,960.0
1883769,30000.0,강원 횡성군,2021-10-30,80.0
1883770,93600.0,강원 횡성군,2021-10-30,260.0


## 산지  
- null 값 제거x  
- 수입산 삭제  
- 시/도 단위로 변환  
- 주산지 선정

In [48]:
rad[rad['SAN_NM'].isnull()]

,TOT_AMT,SAN_NM,SALEDATE,TOT_QTY
17097,307500.0,NaN,2014-06-30,2214.0
17099,2375100.0,NaN,2014-06-30,6786.0
17100,2725500.0,NaN,2014-06-30,7110.0
17101,294000.0,NaN,2014-06-30,1890.0
17116,3465000.0,NaN,2014-06-30,8100.0
...,...,...,...,...
1882899,360000.0,NaN,2021-10-30,800.0
1882900,356400.0,NaN,2021-10-30,792.0
1882901,350000.0,NaN,2021-10-30,800.0
1882902,350000.0,NaN,2021-10-30,800.0


In [49]:
rad['SAN_NM'].unique()

array(['제주도 서귀포시', '제주도 제주시', '전라남도 영암군', '전라북도 고창군', '충청남도 당진군',
       '전라남도 신안군', '전라남도 무안군', '전라남도 해남군', '전라북도 부안군', '충청남도 홍성군',
       '인천광역시 강화군', '경기도 평택시', '경상북도 상주시', '충청북도 음성군', '전라남도 함평군',
       '경기도 고양시', '충청북도 괴산군', '충청남도 예산군', '울산광역시 울주군', '서울특별시 노원구',
       '충청남도 부여군', '광주광역시 광산구', '경기도 김포시', '경상북도 영천시', '서울특별시 송파구',
       '전라북도 군산시', '전라북도 김제시', '경상남도 합천군', '강원도 원주시', '중국', '충청남도 서산시',
       '광주 서구', '전라남도 강진군', '전라남도 나주시', '전라북도 완주군', '전라남도 장성군',
       '서울특별시 강남구', '전라남도 목포시', '전라북도 전주시', '전라남도 영광군', '부산광역시 부산진구',
       '전라북도 정읍시', '경상남도 밀양시', '충청남도 태안군', '충청북도 충주시', '부산광역시 사상구',
       '경기도 양주시', '서울특별시 성북구', '충청남도 논산시', '충청북도 제천시', '경기도 안성시',
       '충청남도 보령시', '경기도 이천시', nan, '강원도 평창군', '경상북도 영덕군', '대전광역시 대덕구',
       '강원도 영월군', '경기도 여주군', '강원도 인제군', '강원도 강릉시', '강원도 홍천군', '강원도 횡성군',
       '경기도 연천군', '강원도 철원군', '경상북도 안동시', '경상북도 봉화군', '경상북도 성주군',
       '전라남도 보성군', '충청남도 아산시', '경기도 포천시', '충청북도 단양군', '전라북도 무주군',
       '경상북도 영양군', '경상북도 청송군', '강원도 정선군', '강원도 삼척

In [50]:
name_split = rad['SAN_NM'].str.split(" ")
rad['SAN_NM'] = name_split.str.get(0)

In [51]:
rad['SAN_NM'].unique()

array(['제주도', '전라남도', '전라북도', '충청남도', '인천광역시', '경기도', '경상북도', '충청북도',
       '울산광역시', '서울특별시', '광주광역시', '경상남도', '강원도', '중국', '광주', '부산광역시', nan,
       '대전광역시', '서울', '제주', '강원', '울산', '부산', '경기', '전남', '충남', '전북',
       '충북', '대구', '경남', '인천', '경북', '대전', '대구광역시', '수입산', '미국', '세종시',
       '원양산', '태국'], dtype=object)

In [52]:
# 수입산 삭제
drop_index = rad[(rad['SAN_NM'] == '중국')|(rad['SAN_NM'] == '수입산')|(rad['SAN_NM'] == '미국')|(rad['SAN_NM'] == '원양산')|(rad['SAN_NM'] == '태국')].index
rad.drop(index = drop_index, axis=0, inplace=True)

In [53]:
rad['SAN_NM'].replace(['경기','서울','전남','충남','강원','경북','전북','경남','충북','광주','제주','대전','부산','대구','세종시','울산','인천'],
                      ['경기도','서울특별시','전라남도','충청남도','강원도','경상북도','전라북도','경상남도','충청북도','광주광역시','제주도','대전광역시','부산광역시','대구광역시','세종특별자치시','울산광역시','인천광역시'],inplace=True)

In [54]:
rad['SAN_NM'].fillna('산지없음',inplace=True)

In [55]:
rad['SAN_NM'].unique()

array(['제주도', '전라남도', '전라북도', '충청남도', '인천광역시', '경기도', '경상북도', '충청북도',
       '울산광역시', '서울특별시', '광주광역시', '경상남도', '강원도', '부산광역시', '산지없음', '대전광역시',
       '대구광역시', '세종특별자치시'], dtype=object)

In [56]:
print(len(rad['SAN_NM'].unique()))

18


In [57]:
rad.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False)/sum(rad.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False)) * 100

SAN_NM
제주도        37.205849
강원도        22.141097
전라북도       11.346218
전라남도        6.636596
충청남도        6.011830
경기도         3.898286
산지없음        2.303549
경상북도        1.951721
경상남도        1.654139
서울특별시       1.453387
광주광역시       1.273251
부산광역시       1.224342
대전광역시       1.053865
충청북도        0.890047
대구광역시       0.532200
울산광역시       0.305803
인천광역시       0.113799
세종특별자치시     0.004020
Name: TOT_QTY, dtype: float64

In [58]:
for i in (2014,2015,2016,2017,2018,2019,2020,2021):
    rad_year = rad[rad['SALEDATE'].dt.year == i]
    joosan = rad_year.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=True).index[-1]
    joosan_amt = rad_year.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=True)[-1]
    ratio = joosan_amt / sum(rad_year.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=True)) * 100
    print(i,'년도', '\n주산지:', joosan, '\n총 물량:', joosan_amt, '\n비율:', round(ratio,2))

2014 년도 
주산지: 제주도 
총 물량: 83095658.0 
비율: 45.66
2015 년도 
주산지: 제주도 
총 물량: 90289114.0 
비율: 47.62
2016 년도 
주산지: 제주도 
총 물량: 172223639.5 
비율: 36.39
2017 년도 
주산지: 제주도 
총 물량: 166765844.90000007 
비율: 33.95
2018 년도 
주산지: 제주도 
총 물량: 145082909.99999997 
비율: 30.5
2019 년도 
주산지: 제주도 
총 물량: 163908315.79999995 
비율: 35.75
2020 년도 
주산지: 제주도 
총 물량: 170158924.10000002 
비율: 35.51
2021 년도 
주산지: 제주도 
총 물량: 156474971.40000004 
비율: 46.57


In [59]:
specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=4, cols=2, specs=specs,
                    subplot_titles=['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'])

fig.add_trace(go.Pie(labels=rad[rad['SALEDATE'].dt.year == 2014].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=rad[rad['SALEDATE'].dt.year == 2014].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2014',), 1, 1)
fig.add_trace(go.Pie(labels=rad[rad['SALEDATE'].dt.year == 2015].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=rad[rad['SALEDATE'].dt.year == 2015].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2015',), 1, 2)
fig.add_trace(go.Pie(labels=rad[rad['SALEDATE'].dt.year == 2016].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=rad[rad['SALEDATE'].dt.year == 2016].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2016',), 2, 1)
fig.add_trace(go.Pie(labels=rad[rad['SALEDATE'].dt.year == 2017].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=rad[rad['SALEDATE'].dt.year == 2017].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2017',), 2, 2)
fig.add_trace(go.Pie(labels=rad[rad['SALEDATE'].dt.year == 2018].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=rad[rad['SALEDATE'].dt.year == 2018].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2018',), 3, 1)
fig.add_trace(go.Pie(labels=rad[rad['SALEDATE'].dt.year == 2019].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=rad[rad['SALEDATE'].dt.year == 2019].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2019',), 3, 2)
fig.add_trace(go.Pie(labels=rad[rad['SALEDATE'].dt.year == 2020].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=rad[rad['SALEDATE'].dt.year == 2020].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2020',), 4, 1)
fig.add_trace(go.Pie(labels=rad[rad['SALEDATE'].dt.year == 2021].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=rad[rad['SALEDATE'].dt.year == 2021].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2021',), 4, 2)

fig.update_traces(hole=.4, hoverinfo='label+percent+name', textinfo='none')
fig.update(layout_title_text='년도별 주산지 비율',
           layout_showlegend=True)

fig = go.Figure(fig)
fig.show()

In [60]:
rad.drop(['SAN_NM'],axis=1,inplace=True)

## 주별로 묶기

In [61]:
rad

,TOT_AMT,SALEDATE,TOT_QTY
0,1456000.0,2014-01-03,5040.0
1,1344000.0,2014-01-03,5040.0
2,1400000.0,2014-01-03,5040.0
3,1344000.0,2014-01-03,5040.0
4,1484000.0,2014-01-03,5040.0
...,...,...,...
1883767,64000.0,2021-10-30,160.0
1883768,336000.0,2021-10-30,960.0
1883769,30000.0,2021-10-30,80.0
1883770,93600.0,2021-10-30,260.0


In [62]:
rad.set_index(['SALEDATE'], inplace=True)

In [63]:
rad_week= rad.resample(rule='W').sum().reset_index()

In [64]:
rad_week

,SALEDATE,TOT_AMT,TOT_QTY
0,2014-01-05,4.405974e+08,1449540.0
1,2014-01-12,9.670760e+08,3884332.0
2,2014-01-19,8.252389e+08,3484982.0
3,2014-01-26,9.860628e+08,3904496.0
4,2014-02-02,7.329970e+08,2881752.0
...,...,...,...
404,2021-10-03,2.682000e+09,9458963.6
405,2021-10-10,2.170946e+09,7344774.8
406,2021-10-17,2.384931e+09,8675149.2
407,2021-10-24,3.934332e+09,8475146.3


In [65]:
rad_week['PER_PRICE'] = rad_week['TOT_AMT']/rad_week['TOT_QTY']

In [66]:
rad_week

,SALEDATE,TOT_AMT,TOT_QTY,PER_PRICE
0,2014-01-05,4.405974e+08,1449540.0,303.956704
1,2014-01-12,9.670760e+08,3884332.0,248.968420
2,2014-01-19,8.252389e+08,3484982.0,236.798612
3,2014-01-26,9.860628e+08,3904496.0,252.545465
4,2014-02-02,7.329970e+08,2881752.0,254.358113
...,...,...,...,...
404,2021-10-03,2.682000e+09,9458963.6,283.540538
405,2021-10-10,2.170946e+09,7344774.8,295.576960
406,2021-10-17,2.384931e+09,8675149.2,274.915266
407,2021-10-24,3.934332e+09,8475146.3,464.219937


In [67]:
rad_week.drop(['TOT_AMT'],axis=1,inplace=True)

In [68]:
fig = px.line(rad_week, x='SALEDATE', y='PER_PRICE')
fig.show()

In [69]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=rad_week['SALEDATE'], y=rad_week['PER_PRICE'], name='단위 당 가격(kg)'))
fig.add_trace(go.Scatter(x=rad_week['SALEDATE'], y=rad_week['TOT_QTY'], name='총 물량'), secondary_y=True)

# Set y-axes titles
fig.update_yaxes(title_text="단위 당 가격(kg)", secondary_y=False)
fig.update_yaxes(title_text="총 물량", secondary_y=True)

fig.update_layout(title_text='총 물량 & 단위 가격')

fig.show()

## 상관관계

In [70]:
corr = rad_week.corr(method = 'pearson')

In [71]:
corr

,TOT_QTY,PER_PRICE
TOT_QTY,1.000000,0.193485
PER_PRICE,0.193485,1.000000


## 정상성 확인(ADF-test)

In [72]:
from statsmodels.tsa.stattools import adfuller 

def adf_test(timeseries):
    result = adfuller(timeseries)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key,value))

In [73]:
adf_test(rad_week['PER_PRICE'])

ADF Statistic: -4.516139
p-value: 0.000184
Critical Values:
	1%: -3.446
	5%: -2.869
	10%: -2.571


In [74]:
adf_test(rad_week['TOT_QTY'])

ADF Statistic: -2.787239
p-value: 0.060120
Critical Values:
	1%: -3.447
	5%: -2.869
	10%: -2.571


In [75]:
from statsmodels.tsa.stattools import adfuller 

def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

In [76]:
# ADF Test on each column
for name, column in rad_week.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "SALEDATE" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.252
 No. Lags Chosen       = 6
 Critical value 1%     = -3.447
 Critical value 5%     = -2.869
 Critical value 10%    = -2.571
 => P-Value = 0.6509. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "TOT_QTY" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.7872
 No. Lags Chosen       = 9
 Critical value 1%     = -3.447
 Critical value 5%     = -2.869
 Critical value 10%    = -2.571
 => P-Value = 0.0601. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "PER_PRICE" 
    -----------------------------------------------
 Null Hypothesis: Data 